In [9]:

class prompt_generator:
    """Generates prompts to infer synthetic data from a chat LLM."""
    
    # the required section of a prompt that generates a patient-gp blood test transcript
    req_sect =  """A patient is having a consultation with a GP. 
    The patient describes symptoms of "{symptoms}"
    Please keep in mind {A}, {B}, {C}, {D}. The doctor has a name and doctor number.
    The questions listed below are being covered in the consultation.
    
    What is the patient's Medicare Card Number
    What is the patient's Last Name
    What are the patient's Given Name/s
    What is the patient's Sex
    What is the patient's Date of Birth
    What is the Reference
    What is the patient's Address 
    What is the patient's Tel(Home)
    What is the patient's Tel(Bus)
    What is the patient's Mobile
    Is the patient Fasting
    Is the patient Non fasting
    Does the patient have a pension card, healthcare concession card, or veteran affairs card?
    {A_1}
    {B_1}
    {C_1}
    {D_1}
   
    The GP makes a determination about the nature of the issue. He then formulates which tests should be run.
    After determining the tests that should be run he asks the patient some follow up questions.
   
    Does the patioent want the billing method to be private, concession or bulkbill?
    
    Write me a transcript of the patient and GP's consultation. Make sure the patient answers all the question points being asked using dummy data. 

    After the consultation the GP writes down the answers to the following questions in a paragraph in free flowing text. He does not ask the patient these questions:
    Who should also receive a copy of the report results
    Are the bloodwork results to be collected by the patient
    What is the doctor number of the requesting doctor
    What is the surname of the Requesting doctor 
    What are the initials of the Requesting doctor 
    What is the address of the Requesting doctor
    Which Hospital/ward is the patient located
    Does patient require copy of this request form?
    What is the doctor number of the requesting doctor
    What is the surname of the requesting doctor 
    What are the initials of the requesting doctor 
    What is the address of the requesting doctor
    Which Hospital/ward is the patient located
    Does patient require copy of this request form?
    Is the bloodwork request URGENT  
    Should the bloodwork be PHONE FAX once complete to me
    if so by what time, and what is my phone fax number
Write the paragraph as if you were the GP
    """
    # the optional sections of the patient-gp blood test transcript
    opt_sect ={
                
                "A":[False,
                     "the doctor {} want the results to be phone/faxed",
                     ""
                    ],
                "B":[False,
                    "the patient {} have a veteran affairs card",
                     "What is the patient's veteran affairs card number"
                    ],
                "C":[False, 
                     "the patient {} have a pension card",
                    "What is the patient's pension card number"
                    ],
                "D":[False,
                     "the patient {} have a Repat Gold card",
                     "What is the patient's Repat Gold card number"
                    ]
                     
    }

    # optional patient symptoms for transcript
    # 1. anemia
    # 2. leukemia
    # 3. leukemia
    # 4. type 2 diabetes
    # 5. gonnorhea
    # 6. liver failure
    symptoms = ["tiredness, and fatigue, and they are a woman.",
                "having a sore body for the past 2 months, and crazy headache." 
                "shoulders hurt first, then femur.",
                "sore body for the last 6 months. My vision was perfect and then it went blurry all of a sudden."
                "Then it worsened in less than a week. Hot showers made me dizzy after a couple minutes in."
                "Walking up the flight of stairs to my apartment was like Mount Everest.",
                "extreme thirst, frequent urination, especially at night,unintentional weight loss, increased hunger"
                ",blurred vision, numbness or tingling in hands and feet, fatigue",
                "swelling in the genital region. They want to get a routine STD test. They are a 25 year old man.",
                "swollen feet, and aches on my sides. I do tend to drink a lot. People say my complexion is becoming more olive."]

    # store concatenated prompts
    prompts = []
    # required prompts for blood test form questions
    req_section2 = """
        From the transcript answer the following questions concisely. The answers must be in a csv format (question, answer). 
        What is the patient's medicare card number
        What is the patient's last name 
        What are the patient's given name/s 
        What is the patient's sex 
        What is the patient's date of birth 
        What is the reference number
        What is the patient's address 
        What is the patient's home telephone number 
        What is the patient's business number 
        What is the patient's mobile number 
        Is the patient fasting
        Is the patient non-fasting  
        Provide a summary of the Clinical Notes about the patient in around 30 words or less  
        Who should also receive a copy of the report results
        Does patient require copy of the blood test request form
        Are the bloodwork results to be collected by the patient
        What is the doctor number of the requesting doctor
        What is the surname of the requesting doctor 
        What are the initials of the requesting doctor 
        What is the address of the requesting doctor
        Which Hospital/ward is the patient located
        Is the bloodwork request urgent
        Should the bloodwork be phone/faxed once complete
        By what time should the bloodwork be phone faxed 
        What is the phone/fax number to send the bloodwork results to
        Is the billing method private  
        Is the billing method concession  
        Is the billing method bulk bill
        What is the patient's vet affairs card number
        What is the patient's healthcare card number
        What is the patient's pension card number
        """
    req_sect2_list = ["What is the patient's medicare card number",
        "What is the patient\'s last name", 
        "What are the patient\'s given name/s", 
        "What is the patient\'s sex", 
        "What is the patient\'s date of birth", 
        "What is the reference number",
        "What is the patient\'s address", 
        "What is the patient\'s home telephone number", 
        "What is the patient\'s business number", 
        "What is the patient\'s mobile number", 
        "Is the patient fasting",
        "Is the patient non-fasting",  
        "Provide a summary of the Clinical Notes about the patient in around 30 words or less",  
        "Who should also receive a copy of the report results",
        "Does patient require copy of the blood test request form",
        "Are the bloodwork results to be collected by the patient",
        "What is the doctor number of the requesting doctor",
        "What is the surname of the requesting doctor", 
        "What are the initials of the requesting doctor", 
        "What is the address of the requesting doctor",
        "Which Hospital/ward is the patient located",
        "Is the bloodwork request urgent",
        "Should the bloodwork be phone/faxed once complete",
        "By what time should the bloodwork be phone faxed", 
        "What is the phone/fax number to send the bloodwork results to",
        "Is the billing method private", 
        "Is the billing method concession",  
        "Is the billing method bulk bill",
        "What is the patient\'s vet affairs card number",
        "What is the patient\'s healthcare card number",
        "What is the patient\'s pension card number"]
        
    def gentext(req_text, opt_sect, symptoms):
        """
        Concatenates optional text to the relevant section in required text.
        
        optional text set to false no text is concatenated.
        optional text set to true text is concatenated.
        """
        format_vals = []
        out_text = ""
    
        format_vals.append({"symptoms":symptoms})
        for i in opt_sect.keys():
            if opt_sect[i][0]:
                format_vals.append({f"{i}":opt_sect[i][1].format("does")})
                format_vals.append({f"{i}_1":opt_sect[i][2]})
                
            else:
                format_vals.append({f"{i}":opt_sect[i][1].format("doesn't")})
                format_vals.append({f"{i}_1":opt_sect[i][2]})
                
        out_text = req_text.format(
            **format_vals[0],**format_vals[1],**format_vals[2],**format_vals[3],\
            **format_vals[4],**format_vals[5],**format_vals[6], **format_vals[7],\
            **format_vals[8]
        )
        return out_text
                
                
    def run():
        """ Runs the text generator for the desired number of combinations."""
        sect_map = {0:"A",1:"B",2:"C",3:"D"}
        curr = 0
        num_opt = 2**len(sect_map)
        text = ""
        
        while curr < num_opt:
            currb = "{0:0"+f"{len(sect_map)}"+"b}"
            currb = currb.format(curr)
            #print(f"iteration: {curr}\n")
            count=0
            for i in currb:
                sect = sect_map[count]
                if i == "1":                
                    prompt_generator.opt_sect[sect][0] = True 
                else:
                    prompt_generator.opt_sect[sect][0] = False
                count+=1
            prompt_generator.prompts.append(prompt_generator.gentext(prompt_generator.req_sect,prompt_generator.opt_sect,prompt_generator.symptoms[curr%6]))
            
            curr+=1
        print(len(prompt_generator.prompts))
        return prompt_generator.prompts

prompt_generator.run()

16
A patient is having a consultation with a GP. 
    The patient describes symptoms of "extreme thirst, frequent urination, especially at night,unintentional weight loss, increased hunger,blurred vision, numbness or tingling in hands and feet, fatigue"
    Please keep in mind the doctor does want the results to be phone/faxed, the patient does have a veteran affairs card, the patient does have a pension card, the patient does have a Repat Gold card. The doctor has a name and doctor number.
    The questions listed below are being covered in the consultation.
    
    What is the patient's Medicare Card Number
    What is the patient's Last Name
    What are the patient's Given Name/s
    What is the patient's Sex
    What is the patient's Date of Birth
    What is the Reference
    What is the patient's Address 
    What is the patient's Tel(Home)
    What is the patient's Tel(Bus)
    What is the patient's Mobile
    Is the patient Fasting
    Is the patient Non fasting
    Does the 

In [ ]:
# store the generated prompts in openai batch format in jsonl
with open("prompts.jsonl", 'w') as file:
    count = 1
    while count < 1024:
        for prompt in prompt_generator.prompts:
                message = {
                    "custom_id": f"request-{count}",
                    "method": "POST",
                    "url": "/v1/chat/completions",
                    "body": {
                        "model": "gpt-4o-mini",
                        "messages": [{"role": "user", "content":prompt}],
                    }
                }
                file.write(json.dumps(message) + "\n")
                count += 1

In [ ]:
# send the batch of prompts to openai model
from openai import OpenAI

prompts = prompt_generator.run()
client = OpenAI(api_key = "...")
batch_input_file= client.files.create(file = open("prompts.jsonl", "rb"), purpose = "batch")
response = client.batches.create(
    input_file_id=batch_input_file.id,
    endpoint = "/v1/chat/completions",
    completion_window = "24h",
    metadata={"description": "GP blood form transcripts"}
)

In [130]:
print(response)

Batch(id='batch_66f3ea484dcc81909fdba559df3b2e33', completion_window='24h', created_at=1727261256, endpoint='/v1/chat/completions', input_file_id='file-rOkL0kjBT7V724d8Fe6vput2', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1727347656, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'GP blood form answers'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [138]:
# retrieve generated transcripts and store
from openai import OpenAI
client = OpenAI(api_key = "...") # api-key

batch = client.batches.retrieve("...") #batch_number
print(batch)
transcripts = client.files.content('...').text # file number

print(text[:1000])

print(client.BatchJob.list())

Batch(id='batch_66f3ea484dcc81909fdba559df3b2e33', completion_window='24h', created_at=1727261256, endpoint='/v1/chat/completions', input_file_id='file-rOkL0kjBT7V724d8Fe6vput2', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1727263195, error_file_id=None, errors=None, expired_at=None, expires_at=1727347656, failed_at=None, finalizing_at=1727263095, in_progress_at=1727261258, metadata={'description': 'GP blood form answers'}, output_file_id='file-85XsnGmOeRPBemASwjdJtu8x', request_counts=BatchRequestCounts(completed=1024, failed=0, total=1024))
{"id": "batch_req_66f3f177e2388190b3dbac154c7096c7", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "c95116e59066a17a052d477e6ca95514", "body": {"id": "chatcmpl-ABJrQrb43Ld4Ivrkqg6zgNuO3JcXU", "object": "chat.completion", "created": 1727261316, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "```csv\n\"question\",\"answer\"

AttributeError: 'OpenAI' object has no attribute 'BatchJob'

In [ ]:
# make a new batch file of prompts to get answers to form questions 
transcripts = transcripts.split("\n")[:-1]

with open("transcripts.jsonl", 'w') as file:
    for transcript in transcripts:
        file.write(transcript)
    file.close()
    
with open("newprompts.jsonl", 'w') as file:
    count = 0
    while count < 1024:
        transcript = transcripts[count].replace("*","")
        transcript = json.loads(transcript)
        transcript = transcript["response"]["body"]["choices"][0]["message"]["content"]
        for prompt in prompt_generator.prompts:
                message = {
                    "custom_id": f"request-{count}",
                    "method": "POST",
                    "url": "/v1/chat/completions",
                    "body": {
                        "model": "gpt-4o-mini",
                        "messages": [{"role": "user", "content":transcript+"\n"+prompt}],
                    }
                }
                file.write(json.dumps(message) + "\n")
                count += 1

In [ ]:
# send the batch of prompts to openai model
from openai import OpenAI

prompts = prompt_generator.run()
client = OpenAI(api_key = "...")
batch_input_file= client.files.create(file = open("newprompts.jsonl", "rb"), purpose = "batch")
response = client.batches.create(
    input_file_id=batch_input_file.id,
    endpoint = "/v1/chat/completions",
    completion_window = "24h",
    metadata={"description": "GP blood form answers"}
)

In [ ]:
print(response)

In [ ]:
# retrieve generated transcripts and store
from openai import OpenAI
client = OpenAI(api_key = "...") # api-key

batch = client.batches.retrieve("...") #batch_number
print(batch)
csv_qa = client.files.content('...').text # file number

print(text[:1000])

print(client.BatchJob.list())

In [107]:
# generate training set 
import csv
count = 0

csv_qa = csv_qa.split("\n")[:-1]
with open('transcripts.jsonl', 'r') as file:
    transcripts = file.read()
transcripts = transcripts.split("\n")

with open("trainsetfull.csv", 'w', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=',',quotechar='"')
    writer.writerow(["question","answer"])
    for count in range(1024):
        qa = csv_qa[count].replace("*","")
        qa = json.loads(qa)
        transcript = transcripts[count].replace("*","")
        transcript = json.loads(transcript)
        transcript = transcript["response"]["body"]["choices"][0]["message"]["content"]

        csv = qa["response"]["body"]["choices"][0]["message"]["content"]
        csv = csv.replace('"',"")
        csv = csv.replace("''","")
        csv = csv.replace("```csv\n","")
        csv = csv.replace("question,answer","")
        csv = csv.replace("```","")
        csv = csv.replace("'","")
        csv = csv.split("\n")
        
        count2 =0
        for count2 in range(len(csv)):
            #print(count2)
            if ("medicare" in csv[count2] or "Medicare" in csv[count2]):
                csv = csv[count2:]
                break
            count2+=1
        count2 =0
        for count2 in range(len(csv)):
            #print(count2)
            if ("pension" in csv[count2]):
                csv = csv[:count2]
                break
        
        for line in range(len(csv)-1):
            #print(line)
            Q = prompt_generator.req_sect2_list[line].strip('"')
            A = csv[line].split(",",1)[1]
            B = csv[line].split(",",1)[0]
            writer.writerow([transcript+"\n\n"+"*"+Q+"*", A])
    file.close()
    
print(f"Wrote {count} transcripts to output")